In [14]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras import layers
from keras.datasets import fashion_mnist, imdb, cifar10, cifar100
from keras.metrics import Precision, Recall, F1Score
from keras.utils import to_categorical, pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Flatten, LSTM, Bidirectional, Dropout, Conv2D, MaxPooling2D, Input, Embedding, SimpleRNN, BatchNormalization
from keras.regularizers import l2, l1
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam, Adamax, Ftrl
from keras.constraints import max_norm
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
def plot_history(history):
    history_dict = history.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    val_acc_values = history_dict['val_accuracy']

    epochs = range(1, len(history_dict['accuracy']) + 1)

    plt.style.use('dark_background')

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss_values, color='lime', label='Training loss')
    plt.plot(epochs, val_loss_values, color='orangered', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(linestyle='--')

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history_dict['accuracy'], color='lime', label='Training acc')
    plt.plot(epochs, val_acc_values, color='orangered', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(linestyle='--')
    
    plt.tight_layout()
    plt.show()

In [16]:
def plot_precision_recall(history):
    precision_values = [history.history[f'precision_class{i}'][-1] for i in range(10)]
    recall_values = [history.history[f'recall_class{i}'][-1] for i in range(10)]

    classes = [str(i) for i in range(10)]
    width = 0.35
    x = range(len(classes))

    plt.figure(figsize=(14, 5))

    plt.bar(x, precision_values, width, label='Precision')
    plt.bar([i + width for i in x], recall_values, width, label='Recall')

    plt.xlabel('Class')
    plt.ylabel('Score')
    plt.title('Precision and Recall for Each Class')
    plt.xticks([i + width/2 for i in x], classes)
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.legend()
    plt.grid(axis='y', linestyle='--')

    plt.show()

In [17]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [18]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train, X_test = X_train / 255., X_test / 255.

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [19]:
precision_per_class = [Precision(class_id=i, name=f'precision_class{i}') for i in range(10)]
recall_per_class = [Recall(class_id=i, name=f'recall_class{i}') for i in range(10)]

all_metrics = ['accuracy'] + precision_per_class + recall_per_class

In [21]:
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.12,
    height_shift_range=0.12,
    shear_range=10,
    channel_shift_range=0.1,
)

train_generator = datagen.flow(X_train, y_train, batch_size=64)

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=45, restore_best_weights=True, start_from_epoch=100, verbose=1)

In [23]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False

In [ ]:
class MyVGGModel():
    def build(self):
        regul_coeff = 0.0001
        
        inputs = Input(shape=(32, 32, 3))
        
        x = Conv2D(64, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(rate=0.1)(x)

        x = Conv2D(128, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = Conv2D(128, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(rate=0.2)(x)

        x = Conv2D(256, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = Conv2D(256, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(rate=0.3)(x)

        x = Conv2D(512, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = Conv2D(512, (3, 3), activation="relu", kernel_regularizer=l2(regul_coeff), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(rate=0.4)(x)

        x = Flatten()(x)
        x = Dense(256, activation="relu")(x)
        outputs = Dense(10, activation="softmax")(x)
        
        model = Model(inputs=inputs, outputs=outputs)

        return model

In [24]:
my_vgg = MyVGGModel()
my_model = my_vgg.build()

In [ ]:
plot_history(history)
plot_precision_recall(history)